# Q4: k-center clustering

## Downloading the dataset

In the following code cell, we download the dataset, which I have uploaded on a public GitHub repository. 

Dataset used: [Credit Card Dataset for Clustering](https://www.kaggle.com/arjunbhasin2013/ccdata)

In [2]:
import requests 

url = "https://raw.githubusercontent.com/frank-chris/CS-328-Assignments/main/HW-1/CC%20GENERAL.csv"

r = requests.get(url) 

with open("CC GENERAL.csv",'wb') as f: 
    f.write(r.content) 

## Reading the csv file into a DataFrame and processing before use



In the following cell, we read the csv file into a DataFrame and set the CUST_ID column as the index.

In [3]:
import pandas as pd
import numpy as np
import random
from itertools import combinations

dataset = pd.read_csv('CC GENERAL.csv')
dataset.set_index('CUST_ID', inplace=True)
dataset.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8949.000000,8950.000000,8637.000000,8950.000000,8950.000000
mean,1564.474828,0.877271,1003.204834,592.437371,411.067645,978.871112,0.490351,0.202458,0.364437,0.135144,3.248827,14.709832,4494.449450,1733.143852,864.206542,0.153715,11.517318
std,2081.531879,0.236904,2136.634782,1659.887917,904.338115,2097.163877,0.401371,0.298336,0.397448,0.200121,6.824647,24.857649,3638.815725,2895.063757,2372.446607,0.292499,1.338331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.019163,0.000000,6.000000
25%,128.281915,0.888889,39.635000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,383.276166,169.123707,0.000000,12.000000
50%,873.385231,1.000000,361.280000,38.000000,89.000000,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,856.901546,312.343947,0.000000,12.000000
75%,2054.140036,1.000000,1110.130000,577.405000,468.637500,1113.821139,0.916667,0.300000,0.750000,0.222222,4.000000,17.000000,6500.000000,1901.134317,825.485459,0.142857,12.000000
max,19043.138560,1.000000,49039.570000,40761.250000,22500.000000,47137.211760,1.000000,1.000000,1.000000,1.500000,123.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000,12.000000


In the nest cell, we find the number of rows having NaNs.

In [4]:
dataset.shape[0] - dataset.dropna().shape[0]

314

In the next cell, we find the columns that have NaNs

In [5]:
dataset.columns[dataset.isna().any()].tolist()

['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']

In the next cell, we find the total number of NaN cells in the dataset.

In [6]:
sum(dataset.isnull().values.ravel())

314

In the next cell, we fill the NaN cells with the mean of the corresponding column. Using the mean will not cause any deviations in the data and is better than filling with zeroes. Another option is to remove these points at the cost of loss of information.

In [7]:
dataset.fillna(dataset.mean(), inplace=True)

In the next cell, we perform min-max scaling normalisation to make sure values in all columns are in the range between 0 and 1.

In [8]:
dataset = (dataset-dataset.min())/(dataset.max()-dataset.min())
dataset.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000
mean,0.082154,0.877271,0.020457,0.014534,0.018270,0.020766,0.490351,0.202458,0.364437,0.090096,0.026413,0.041089,0.148396,0.034170,0.011310,0.153715,0.919553
std,0.109306,0.236904,0.043570,0.040722,0.040193,0.044491,0.401371,0.298336,0.397448,0.133414,0.055485,0.069435,0.121490,0.057078,0.030503,0.292499,0.223055
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.006736,0.888889,0.000808,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.002793,0.051753,0.007556,0.002236,0.000000,1.000000
50%,0.045864,1.000000,0.007367,0.000932,0.003956,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,0.019553,0.098497,0.016894,0.004392,0.000000,1.000000
75%,0.107868,1.000000,0.022637,0.014166,0.020828,0.023629,0.916667,0.300000,0.750000,0.148148,0.032520,0.047486,0.215359,0.037482,0.011310,0.142857,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Implementing algorithms

dist(a, b) gives the L2 norm of a-b, and is the distance function used.

In [9]:
def dist(a, b):
  return np.linalg.norm(a-b)

find_cost(dataset, C) returns the total cost of clustering when C is the list of centers

In [10]:
def find_cost(dataset, C):
  clusters = [[] for i in range(len(C))]
  for index, row in dataset.iterrows():
    dist_to_closest_center = dist(C[0], row)
    closest_center = 0
    for i, c in enumerate(C):
      if dist(c, row) <= dist_to_closest_center:
        dist_to_closest_center = dist(c, row)
        closest_center = i
    clusters[closest_center].append(dist_to_closest_center)

  cluster_costs = [max(point_costs) for point_costs in clusters]
  cost = max(cluster_costs)
  return cost

min_dist_from_centers(x, C) returns the minimum among the distances of the point x from the centers.

In [11]:
def min_dist_from_centers(x, C):
  min_dist = dist(x, C[0])
  for c in C:
    if dist(x, c) < min_dist:
      min_dist = dist(x, c)
  return min_dist

next_center(dataset, C) returns the next center to be chosen when C is the list centers chosen so far.

In [12]:
def next_center(dataset, C):
  next = dataset.iloc[0]
  for index, row in dataset.iterrows():
    if min_dist_from_centers(row, C) > min_dist_from_centers(next, C):
      next = row
  return next

k_center_approx(dataset, k) performs approximate k-center clustering of dataset and returns the list of centers (C) and the total cost of clustering (cost) as a tuple.

In [13]:
def k_center_approx(dataset, k):
  C = []
  C.append(dataset.iloc[random.randint(0, dataset.index.size - 1)])
  for i in range(k-1):
    C.append(next_center(dataset, C))

  cost = find_cost(dataset, C)
  return C, cost

k_center_exact(dataset, k) performs exact(optimal) k-center clustering of dataset and returns the list of centers (C_optimal) and the total cost (cost_optimal) of clustering as a tuple.

In [14]:
def k_center_exact(dataset, k):
  C_optimal = []
  cost_optimal = np.inf
  for index in list(combinations(dataset.index,k)):
    C = []
    for i, row in dataset.loc[index,:].iterrows():
      C.append(row)
    if find_cost(dataset, C) < cost_optimal:
      C_optimal = C
      cost_optimal = find_cost(dataset, C)
  return C_optimal, cost_optimal
    

Reporting the k-center objective value for k = 2, 4, 10 using approximate algorithm(greedy).

In [15]:
k_list = [2, 4, 10]

for k in k_list:
  C, cost = k_center_approx(dataset, k)
  
  print('Cost(objective value) for k =', k, 'is', cost)

Cost(objective value) for k = 2 is 2.051034307060489
Cost(objective value) for k = 4 is 1.7659087627465788
Cost(objective value) for k = 10 is 1.4892928394430194


Printing the centers for the clustering performed above.

In [16]:
for k in k_list:
  print('Centers for k =', k, ':\n')
  for i, c in enumerate(C):
    print('c -', i+1, ':\n')
    print(c, '\n')

Centers for k = 2 :

c - 1 :

BALANCE                             0.004727
BALANCE_FREQUENCY                   0.909091
PURCHASES                           0.025436
ONEOFF_PURCHASES                    0.010476
INSTALLMENTS_PURCHASES              0.036461
CASH_ADVANCE                        0.000000
PURCHASES_FREQUENCY                 0.909091
ONEOFF_PURCHASES_FREQUENCY          0.272727
PURCHASES_INSTALLMENTS_FREQUENCY    0.818182
CASH_ADVANCE_FREQUENCY              0.000000
CASH_ADVANCE_TRX                    0.000000
PURCHASES_TRX                       0.069832
CREDIT_LIMIT                        0.165275
PAYMENTS                            0.015040
MINIMUM_PAYMENTS                    0.001505
PRC_FULL_PAYMENT                    0.454545
TENURE                              0.833333
Name: C15304, dtype: float64 

c - 2 :

BALANCE                             0.606387
BALANCE_FREQUENCY                   1.000000
PURCHASES                           1.000000
ONEOFF_PURCHASES              

Finding the optimal cost and reporting the approximation factor obtained by the greedy algorithm for k = 2, 4 by only taking 20 (this number can be changed in the following cell) points randomly and uniformly from the dataset.

In [17]:
k_list = [2, 4]

subset = dataset.sample(n=20)

for k in k_list:
  C_approx, cost_approx = k_center_approx(subset, k)
  C_optimal, cost_optimal = k_center_exact(subset, k)

  print('Cost(approximate) for k =', k, 'is', cost_approx)
  print('Cost(optimal) for k =', k, 'is', cost_optimal)
  print('Approximation factor for k =', k, 'is', cost_approx/cost_optimal, '\n')

Cost(approximate) for k = 2 is 1.5333044038552515
Cost(optimal) for k = 2 is 1.13214981728322
Approximation factor for k = 2 is 1.354329948605802 

Cost(approximate) for k = 4 is 1.0069456800092873
Cost(optimal) for k = 4 is 0.9791892330772924
Approximation factor for k = 4 is 1.0283463563470412 

